In [1]:
from pyspark.sql import SparkSession, functions as F, types as T

In [2]:
spark = SparkSession.builder.master("spark://spark-master:7077").config("spark.jars.packages", "org.apache.hadoop:hadoop-aws-2.7.3").appName("spark-hw").getOrCreate()
#spark = SparkSession.builder.master("local[*]").getOrCreate()

25/06/22 15:49:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
print(spark.version)

3.5.5


In [4]:
actor_df = spark.read.csv('../data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('../data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('../data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('../data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('../data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('../data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('../data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('../data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('../data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('../data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('../data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('../data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('../data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('../data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('../data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [5]:
film_counts = film_category_df.groupBy("category_id").count()

result = film_counts.join(category_df, "category_id") \
    .select("name", "count") \
    .orderBy("count", ascending=False)

result.show()


+-----------+-----+
|       name|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [6]:
rental_with_film = rental_df.join(inventory_df, "inventory_id")
rental_with_actor = rental_with_film.join(film_actor_df, "film_id")
rental_with_names = rental_with_actor.join(actor_df, "actor_id")

top_actors = (
    rental_with_names
    .withColumn("full_name", F.concat_ws(" ", "first_name", "last_name"))
    .groupBy("full_name")
    .count()
    .orderBy("count", ascending=False)
    .limit(10)
)

top_actors.show()

+------------------+-----+
|         full_name|count|
+------------------+-----+
|       SUSAN DAVIS|  825|
|    GINA DEGENERES|  753|
|    MATTHEW CARREY|  678|
|       MARY KEITEL|  674|
|ANGELA WITHERSPOON|  654|
|       WALTER TORN|  640|
|       HENRY BERRY|  612|
|       JAYNE NOLTE|  611|
|        VAL BOLGER|  605|
|     SANDRA KILMER|  604|
+------------------+-----+



3.
Вивести категорію фільмів, на яку було витрачено найбільше грошей
в прокаті

In [7]:
rental_with_film = rental_df.join(inventory_df, "inventory_id")
rental_with_film_data = rental_with_film.join(film_df, "film_id")
rental_with_category = rental_with_film_data.join(film_category_df, "film_id")
rental_with_category_name = rental_with_category.join(category_df, "category_id")

category_spending = (
    rental_with_category_name
    .groupBy("name")
    .agg(F.sum("rental_rate").alias("total_spent"))
    .orderBy("total_spent", ascending=False)
    .limit(1)
)

category_spending.show()


+------+------------------+
|  name|       total_spent|
+------+------------------+
|Sports|3617.2099999998945|
+------+------------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [8]:
films_not_in_inventory = (
    film_df
    .join(inventory_df, on="film_id", how="left_anti")
    .select("title")
)

films_not_in_inventory.show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [9]:
children_category = category_df.filter(category_df.name == "Children")

children_films = film_category_df.join(children_category, "category_id")

actors_in_children = film_actor_df.join(children_films, "film_id")

actors_named = actors_in_children.join(actor_df, "actor_id") \
    .withColumn("full_name", F.concat_ws(" ", "first_name", "last_name"))

top_3 = (
    actors_named
    .groupBy("full_name")
    .count()
    .orderBy("count", ascending=False)
    .limit(3)
)

top_3.show()

+------------+-----+
|   full_name|count|
+------------+-----+
|HELEN VOIGHT|    7|
| SUSAN DAVIS|    6|
|  MARY TANDY|    5|
+------------+-----+



Stop Spark session:

In [10]:
spark.stop()